<a href="https://colab.research.google.com/github/dAn-solution/competition/blob/main/Prob_kiva_025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kiva／クラウドファンディングの資金調達額予測
- ランダムフォレスト を実施

### Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


### ライブラリのインストール、インポート

In [ ]:
# カレントディレクトリを変更
import os
os.chdir('/content/drive/My Drive/Probdata/kiva/')
print(os.getcwd())

/content/drive/My Drive/Probdata/kiva


In [ ]:
class Config():
    root_path = './'
    input_path = os.path.join(root_path, 'input')
    output_path = os.path.join(root_path, 'output')
    result_path = os.path.join(root_path, 'result')
    bert_model_name = 'bert-base-uncased'
    seed = 42
    debug = False

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor as RFR

### データの読み込み

In [ ]:
train_df = pd.read_csv(f'{Config.input_path}/train.csv')
test_df = pd.read_csv(f'{Config.input_path}/test.csv')
sample_df = pd.read_csv(f'{Config.input_path}/sample_submission.csv')

train_011_df = pd.read_csv(f'{Config.result_path}/train_011.csv')
test_011_df = pd.read_csv(f'{Config.result_path}/test_011.csv')
train_012_df = pd.read_csv(f'{Config.result_path}/train_012.csv')
test_012_df = pd.read_csv(f'{Config.result_path}/test_012.csv')
# train_013_df = pd.read_csv(f'{Config.result_path}/train_013.csv')
# test_013_df = pd.read_csv(f'{Config.result_path}/test_013.csv')
# train_014_df = pd.read_csv(f'{Config.result_path}/train_014.csv')
# test_014_df = pd.read_csv(f'{Config.result_path}/test_014.csv')
train_015_df = pd.read_csv(f'{Config.result_path}/train_015.csv')
test_015_df = pd.read_csv(f'{Config.result_path}/test_015.csv')
train_016_df = pd.read_csv(f'{Config.result_path}/train_016.csv')
test_016_df = pd.read_csv(f'{Config.result_path}/test_016.csv')
train_020_df = pd.read_csv(f'{Config.result_path}/train_020.csv')
test_020_df = pd.read_csv(f'{Config.result_path}/test_020.csv')

## ランダムフォレストの実行

In [ ]:
target_column = "LOAN_AMOUNT"
y = train_df[target_column]

In [ ]:
# 合体
# X = pd.concat([train_011_df, train_012_df, train_013_df, train_014_df, train_015_df, train_016_df, train_020_df], axis=1)
# test_X = pd.concat([test_011_df, test_012_df, test_013_df, test_014_df, test_015_df, test_016_df, test_020_df], axis=1)
X = pd.concat([train_011_df, train_012_df, train_015_df, train_016_df, train_020_df], axis=1)
test_X = pd.concat([test_011_df, test_012_df, test_015_df, test_016_df, test_020_df], axis=1)

In [ ]:
# trainデータを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = Config.seed)

In [ ]:
# 学習/予測

y_oof = np.zeros(len(y_train))
meta_preds = []
y_preds = []
kf = StratifiedKFold(n_splits=5)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
    x_tr_fold = X.iloc[tr_idx]
    y_tr_fold = y.iloc[tr_idx]
    x_vl_fold = X.iloc[vl_idx]
    y_vl_fold = y.iloc[vl_idx]

    model = RFR(n_estimators=10)
    model.fit(x_tr_fold, y_tr_fold)
    # model.fit(
    #     x_tr_fold, y_tr_fold,
    #     eval_set=(x_vl_fold, y_vl_fold),
    #     eval_metric='l1',
    #     verbose=False,
    #     early_stopping_rounds=100,
    # )
    y_oof[vl_idx] = model.predict(x_vl_fold)
    meta_preds.append(model.predict(X))
    y_preds.append(model.predict(test_X))
    print(
        f'fold {fold} score:', mean_absolute_error(y_oof[vl_idx], y_vl_fold)
    )

print('X score:', mean_absolute_error(model.predict(X), y))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


fold 0 score: 135.4175790338032
fold 1 score: 132.57818380893724
fold 2 score: 131.68976254020393
fold 3 score: 131.40234722507356
fold 4 score: 134.47290084171627
X score: 125.34631513253699


In [ ]:
# メタモデルへの引き渡し
train_025_df = pd.DataFrame(np.mean(meta_preds, axis=0))
train_025_df.to_csv(os.path.join(Config.result_path, "train_025.csv"), index=False)

test_025_df = pd.DataFrame(np.mean(y_preds, axis=0))
test_025_df.to_csv(os.path.join(Config.result_path, "test_025.csv"), index=False)

### 提出データの作成

In [ ]:
sample_df["LOAN_AMOUNT"] = np.mean(y_preds, axis=0)
print(sample_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445        480.0
1      1944435        586.5
2      2083354        361.5
3      1993565        775.5
4      2064272        200.0
...        ...          ...
91817  1993862        346.0
91818  2015070        221.0
91819  1950349        712.5
91820  1921580       1546.0
91821  1976733        523.0

[91822 rows x 2 columns]


In [ ]:
# submitファイルのファイル名に利用する作成時刻の取得

now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1)))
date_time = "{0:%Y%m%d_%H%M}".format(now)

In [ ]:
# submitファイルの出力(Google Driveに出力）
# 形式：submit_yyyymmdd_hhmm.csv

sample_df.to_csv(os.path.join(Config.output_path, f"submit_{date_time}.csv"), index=False)


In [ ]:
test_025_df.describe()

,0
count,91822.000000
mean,642.665314
std,603.669555
min,50.000000
25%,338.500000
50%,472.500000
75%,726.000000
max,9606.500000
